<a href="https://colab.research.google.com/github/Mobeenahmedmehr/Mobeen_Ahmed/blob/main/RAG_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installing Required Libraries

In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


# What it does:
## This line installs two Python libraries:

* **langchain-pinecone**: This library helps in working with Pinecone, a vector
database used for storing and searching embeddings (numerical representations of text).

* **langchain-google-genai**: This library provides tools to interact with Google's Generative AI models, like embeddings and chat models.





# 2. Setting Up Pinecone

In [6]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONEAPIKEY')

pc = Pinecone(api_key=pinecone_api_key)

## What it does:

* **userdata.get('PINECONEAPIKEY')**: This retrieves your Pinecone API key from Google Colab's user data. You need to store your API key in Colab's secrets manager.

* **Pinecone(api_key=pinecone_api_key)**: This initializes the Pinecone client using your API key, allowing you to interact with Pinecone's services.

# **3. Creating a Pinecone Index**


In [8]:
index_name = "simple-rag-project"  # change if desired

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

## What it does:

* **pc.create_index(...)**: This creates a new index in Pinecone. An index is like a database table where you store vectors (embeddings).

* **name**: The name of the index.

* **dimension=768**: The size of the vectors you'll store. Here, each vector has 768 dimensions.

* **metric="cosine"**: The similarity metric used to compare vectors. Cosine similarity is a common choice for text embeddings.

* **spec=ServerlessSpec(...)**: This specifies that the index will be hosted on AWS in the us-east-1 region.

* **index = pc.Index(index_name)**: This connects to the newly created index.

# 4. Setting Up Google Generative AI Embeddings

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('Gemini_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

## What it does:

🌸**os.environ['GOOGLE_API_KEY']**: This sets the Google API key in the environment variables, which is required to use Google's Generative AI services.

🌸**GoogleGenerativeAIEmbeddings(model="models/embedding-001")**: This initializes the embeddings model, which converts text into vectors (embeddings).

# 5. Creating an Embedding

In [10]:
vector = embeddings.embed_query("We are building a RAG Text")

In [11]:
vector[:5]

[0.03732863813638687,
 -0.054927337914705276,
 -0.05723658204078674,
 -0.016060668975114822,
 0.008930283598601818]

## What it does:

🌸**embeddings.embed_query(...)**: This converts the text "We are building a RAG Text" into a vector (embedding).

🌸**vector[:5]**: This shows the first 5 values of the vector. Vectors are just lists of numbers that represent the text in a way that computers can understand.

# 6. Setting Up Pinecone Vector Store

In [12]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

## What it does:

🌸**PineconeVectorStore(...)**: This sets up a vector store, which is a way to store and search vectors in Pinecone. It uses the index we created earlier and the embeddings model.

# 7. Creating Documents

In [14]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)
document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [15]:
len(documents)

10

## What it does:

🌸**Document(...)**: This creates a document object that contains text (page_content) and metadata (like the source of the text).

🌸**documents**: This is a list of all the documents you want to store in the vector store.

# 8. Adding Documents to the Vector Store

In [16]:
from uuid import uuid4
uuid4()

UUID('85dd2a19-605d-43f7-bdf1-49dffd2d174d')

In [20]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)


['57919327-463b-4ff5-93dc-26f9f0bfa558',
 'af4fc029-b8f8-4a02-a4d3-d70711d595da',
 '8276be3a-72f0-490f-9663-448e0d950fa4',
 '20cced97-28d7-492a-ba56-852770ea9c6e',
 'f3073b3a-c5be-4a78-b61e-25e7ba6d62d8',
 'a31f5233-56f0-4627-9fda-d93b15b7b336',
 'db4c2044-b46c-4ce9-a833-84bb37aa0c04',
 '51cc108b-5a0e-4e8c-a0e5-b05ecd4ce11e',
 '8794c5b2-0d40-47c8-8c07-8b0baf271f78',
 '9141da1c-781d-4d38-b11a-163735c55081']

## What it does:

🌸**uuid4()**: This generates a unique identifier (UUID) for each document.

🌸**vector_store.add_documents(...)**: This adds the documents to the vector store, associating each document with a unique ID.

# 9. Searching for Similar Documents

In [21]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [22]:
results = vector_store.similarity_search_with_score(
    "How to bake Cookies?",

)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

## What it does:

🌸**similarity_search(...)**: This searches for documents that are similar to the query "LangChain provides abstractions to make working with LLMs easy".

🌸**k=2**: It returns the top 2 most similar documents.

🌸**filter={"source": "tweet"}**: It only considers documents that have a "source" metadata value of "tweet".

The **results** are printed, showing the content and metadata of the similar documents

# 10. Using a Chat Model for Answering Questions

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## What it does:

🌸**ChatGoogleGenerativeAI(...)**: This initializes a chat model from Google's Generative AI. It will be used to generate answers based on the user's query and the documents found in the vector store.

# 11. Answering User Queries

In [24]:
def answer_to_user(query: str):

  # Vector Search
  vector_results = vector_store.similarity_search(query, k=2)
  print(len(vector_results))

  # TODO: Pass to Model Vector Results + User Query
  final_answer = llm.invoke(f"ANSWER THIS USER QUERY: {query}, Here are some references to answer {vector_results}")

  return final_answer

## What it does:

🌸**vector_store.similarity_search(query, k=2)**: This searches for the top 2 documents similar to the user's query.

🌸**llm.invoke(...)**: This sends the user's query and the similar documents to the chat model, asking it to generate an answer based on the provided information.

# 12. Example Usage

In [25]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")

2


In [26]:
answer.content

"Based on the provided documents, LangChain is a framework used for building applications, and one user is excited about a new project built with it.  The documents don't directly support the claim that LangChain provides abstractions to make working with LLMs easy, although that is a common understanding of LangChain's functionality.  The first document mentions LangGraph, a different framework, suggesting there are multiple options for building applications involving LLMs."

## What it does:

🌸This calls the **answer_to_user** function with a specific query and prints the generated answer.